# Streaming

This notebook covers functionality related to streaming.

For more information, see:

- [Streaming with LCEL](https://python.langchain.com/docs/expression_language/interface#stream)

- [Streaming for RAG](https://python.langchain.com/docs/use_cases/question_answering/streaming)

- [Streaming for Agents](https://python.langchain.com/docs/modules/agents/how_to/streaming)

## Basic Streaming

In [5]:
import logging
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain = prompt | model | output_parser

In [3]:
for s in chain.stream({"topic": "bears"}):
    print(s)


Why
 don
't
 bears
 use
 cell
 phones
?


Because
 they
 can
't
 bear
 to
 have
 their
 calls
 dropped
!


## Streaming with RunnableParallel

In [4]:
from langchain_core.runnables import RunnableParallel

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain1 = prompt | model | output_parser

prompt = ChatPromptTemplate.from_template("Write me a poem about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()
chain2 = prompt | model | output_parser

parallel_chain = RunnableParallel({
    "joke": chain1,
    "poem": chain2
})

for s in parallel_chain.stream({"topic": "bears"}):
    print(s)

{'poem': ''}
{'poem': 'In'}
{'poem': ' the'}
{'poem': ' wild'}
{'poem': ' where'}
{'poem': ' nature'}
{'poem': "'s"}
{'poem': ' beauty'}
{'poem': ' thr'}
{'poem': 'ives'}
{'poem': ',\n'}
{'poem': 'Am'}
{'poem': 'id'}
{'poem': 'st'}
{'poem': ' the'}
{'poem': ' towering'}
{'joke': ''}
{'poem': ' trees'}
{'poem': ' and'}
{'poem': ' buzzing'}
{'joke': 'Why'}
{'joke': ' don'}
{'joke': "'t"}
{'poem': ' h'}
{'poem': 'ives'}
{'poem': ',\n'}
{'poem': 'A'}
{'poem': ' creature'}
{'poem': ' ro'}
{'poem': 'ams'}
{'poem': ' with'}
{'poem': ' majestic'}
{'joke': ' bears'}
{'joke': ' wear'}
{'joke': ' shoes'}
{'joke': '?\n\n'}
{'joke': 'Because'}
{'joke': ' they'}
{'joke': ' have'}
{'joke': ' bear'}
{'joke': ' feet'}
{'joke': '!'}
{'joke': ''}
{'poem': ' flair'}
{'poem': ',\n'}
{'poem': 'The'}
{'poem': ' mighty'}
{'poem': ' bear'}
{'poem': ','}
{'poem': ' beyond'}
{'poem': ' compare'}
{'poem': '.\n\n'}
{'poem': 'With'}
{'poem': ' fur'}
{'poem': ' as'}
{'poem': ' dark'}
{'poem': ' as'}
{'poem': ' the'}

In [7]:
result = {}
for s in parallel_chain.stream({"topic": "bears"}):
    for k,v in s.items():
        if k not in result:
            result[k] = ""
        result[k] += v
    print(result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'poem': ''}
{'poem': 'In'}
{'poem': 'In the'}
{'poem': 'In the wild'}
{'poem': 'In the wild,'}
{'poem': 'In the wild, where'}
{'poem': 'In the wild, where nature'}
{'poem': "In the wild, where nature's"}
{'poem': "In the wild, where nature's beauty"}
{'poem': "In the wild, where nature's beauty thr"}
{'poem': "In the wild, where nature's beauty thrives"}
{'poem': "In the wild, where nature's beauty thrives,\n"}
{'poem': "In the wild, where nature's beauty thrives,\nRes"}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'poem': "In the wild, where nature's beauty thrives,\nResides"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle lives"}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle lives", 'joke': ''}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle lives,\n", 'joke': ''}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle lives,\nA", 'joke': ''}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle lives,\nA majestic", 'joke': ''}
{'poem': "In the wild, where nature's beauty thrives,\nResides a creature with gentle

## Stream Log

In [8]:
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever
from langchain_core.runnables import RunnablePassthrough

retriever= TavilySearchAPIRetriever()

prompt = ChatPromptTemplate.from_template("""Answer the question based only on the context provided:

Context: {context}

Question: {question}""")

chain = prompt | model | output_parser

retrieval_chain = RunnablePassthrough.assign(
    context=(lambda x: x["question"]) | retriever.with_config(run_name="Docs")
) | chain

In [9]:
for s in retrieval_chain.stream({"question": "what is langsmith"}):
    print(s, end="")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


LangSmith is a platform that helps trace and evaluate language model applications and intelligent agents. It allows developers to move from prototyping to production by providing tools for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith also offers a simple and intuitive user interface that reduces the barrier to entry for those without a software background.

In [10]:
async for s in retrieval_chain.astream_log({"question": "what is langsmith"}):
    print(s, end="")

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '103db7cb-3ff7-495f-a862-97f4edfb9b7b',
            'logs': {},
            'streamed_output': []}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<context>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '0a0ca07b-8229-4e7d-8aa9-3b752cba3dd5',
            'metadata': {},
            'name': 'RunnableAssign<context>',
            'start_time': '2024-01-14T11:04:46.434+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:1'],
            'type': 'chain'}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<context>/streamed_output/-',
  'value': {'question': 'what is langsmith'}})RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel<context>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '99135217-b8a1-4e3a-8bc8-5b7fac668780',
            '

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RunLogPatch({'op': 'add',
  'path': '/logs/StrOutputParser',
  'value': {'end_time': None,
            'final_output': None,
            'id': '0223ff38-6fc7-4071-a82b-f39c124a19e1',
            'metadata': {},
            'name': 'StrOutputParser',
            'start_time': '2024-01-14T11:04:49.283+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:4'],
            'type': 'parser'}})RunLogPatch({'op': 'add', 'path': '/logs/StrOutputParser/streamed_output/-', 'value': ''})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ''},
 {'op': 'replace', 'path': '/final_output', 'value': ''})RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='')})RunLogPatch({'op': 'add',
  'path': '/logs/StrOutputParser/streamed_output/-',
  'value': 'Lang'})RunLogPatch({'op': 'add', 'path': '/streame

In [11]:
async for s in retrieval_chain.astream_log({"question": "what is langsmith"}, include_names=["Docs"]):
    print(s, end="")

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '559e6038-da33-4a31-9eda-e0ae2ff42768',
            'logs': {},
            'streamed_output': []}})RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '791f30f5-ca81-4244-81b2-00c829ba6be9',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-01-14T11:06:31.322+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2'],
            'type': 'retriever'}})RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='LangSmith helps you trace and evaluate your language model applications and intelligent agents to help you move from prototype to production. Check out the interactive walkthrough to get started. For more information, please refer to the LangSmith documentation

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ''},
 {'op': 'replace', 'path': '/final_output', 'value': ''})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': 'Lang'},
 {'op': 'replace', 'path': '/final_output', 'value': 'Lang'})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': 'Smith'},
 {'op': 'replace', 'path': '/final_output', 'value': 'LangSmith'})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ' is'},
 {'op': 'replace', 'path': '/final_output', 'value': 'LangSmith is'})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ' a'},
 {'op': 'replace', 'path': '/final_output', 'value': 'LangSmith is a'})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ' platform'},
 {'op': 'replace', 'path': '/final_output', 'value': 'LangSmith is a platform'})RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ' that'},
 {'op': 'replace',
  'path': '/final_output',
  'value': 'LangSmith is a platf

## Agents

### Stream Actions

In [14]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [search]

# Get the prompt to use - you can modify this!
# If you want to see the prompt in full, you can at: https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [15]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk)
    print("------")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}})])], 'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}})]}
------
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in Los Angeles'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in Los Angeles'}`\nresponded: The current weather information for San Francisco is not explicitly detailed in the search results. However, you can find up-to-date weather information for San Francisco by visiting [Weather25.com](https://www.weather25.com/north-america/usa/california/san-francisco) or checking climate averages and historical data on [Climate-Data.org](https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/).\n\nNow, let's check the weather in Los Angeles.\n\n", message_log=[AIMessage(content="The current weather information for San Francisco is not explicitly detailed in the search results. However, you can find up-to-date weather information for San Francisco by visiting [Weather25.com](https://www.weather25.com/nor

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'output': 'The search results do not provide a direct current temperature for Los Angeles, but they do offer some historical data and forecasts. For instance, on January 14, 2024, the weather forecast predicted a maximum temperature of 17°C (62°F) and a minimum of 11°C (51°F) for Los Angeles.\n\nFor the most accurate and current weather conditions in Los Angeles, you may want to visit a dedicated weather website or use a weather app.', 'messages': [AIMessage(content='The search results do not provide a direct current temperature for Los Angeles, but they do offer some historical data and forecasts. For instance, on January 14, 2024, the weather forecast predicted a maximum temperature of 17°C (62°F) and a minimum of 11°C (51°F) for Los Angeles.\n\nFor the most accurate and current weather conditions in Los Angeles, you may want to visit a dedicated weather website or use a weather app.')]}
------


### Stream Tokens

In [16]:
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, streaming=True)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [17]:
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '271903a5-b404-4a9f-a961-8ec427f4cf89',
            'logs': {},
            'streamed_output': []}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI',
  'value': {'end_time': None,
            'final_output': None,
            'id': '88cb21f2-60d7-40d4-94cb-4b31bb6bc3dc',
            'metadata': {},
            'name': 'ChatOpenAI',
            'start_time': '2024-01-14T11:31:26.694+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'tavily_search_results_json'}})})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"', 'name': ''}})})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': 'query', 'name': ''}})})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(conte

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI:2/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': 'The'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content='The')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' current'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content=' current')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' weather'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-',
  'value': AIMessageChunk(content=' weather')})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output_str/-',
  'value': ' in'},
 {'op': 'add',
  'path': '/logs/ChatOpenAI:2/streamed_output/-'